In [1]:
from N1_dataprep_mnist import *

In [2]:
# ML Part
test_ratio = 0.25
epochs = 2
count = 7

train_inp, test_inp, train_oup, test_oup = split(fetch(), test_ratio)
train_inps = buildPortions(count, train_inp)
train_oups = buildPortions(count, train_oup)
test_inps = buildPortions(count, test_inp)
test_oups = buildPortions(count, test_oup)

[26456, 13228, 6614, 3307, 1653, 826, 413]
[26456, 13228, 6614, 3307, 1653, 826, 413]
[8818, 4409, 2204, 1102, 551, 275, 137]
[8818, 4409, 2204, 1102, 551, 275, 137]


In [3]:
def honestTrainGlobal(epochs, train_inp_, train_oup_):
    sess = getSession()
    md = CNN()
    md.setup()
    md.setData(train_inp_,train_oup_)
    md.train('FMN', 'Global-' + sess, epochs_=epochs, verbose_=2)
    return md

def honestGlobal(test_inp_, test_oup_, model_):
    model_.model.evaluate(test_inp_, test_oup_)

def clientsEval(test_inps_, test_oups_):
    testMulti(test_inps_, test_oups_)

def randomize(train_oup__):
    train_oup_ = deepcopy(train_oup__)
    for i in range(len(train_oup_)):
        for j in range(len(train_oup_[i])):
            deviation = randint(0,300)/100
            train_oup_[i][j] = train_oup[i][j] * deviation
    print(train_oup__ - train_oup_)
    return train_oup_    

def shuffle(lst):
    lst_ = []
    for i in range(len(lst)):
        candidate = randint(-1,1)
        if lst[i] == 1 and lst[i] == candidate:
            candidate = 0
        lst_.append(candidate)
    return lst_

def poison(seth, defect_ratio):
    seth_ = deepcopy(seth)
    sumh = 0
    altered = 0
    for e in seth_:
        sumh += len(e)
    for i in range(len(seth_)):
        ratio = len(seth_[i])/sumh
        if ratio > defect_ratio:
            continue
        else:
            #train_oups_[i] = randomize(train_oups_[i])
            for j in range(len(seth_[i])):
                seth_[i][j] = shuffle(seth_[i][j])
                altered += 1
            defect_ratio -= ratio
    print("Altered: " + str(altered) + " of " + str(sumh))
    return seth_

def dishonestTrain(epochs, train_inps_, train_oups_):
    train_inp__ = []
    train_oup__ = []
    for i in range(len(train_inps_)):
        train_inp__.extend(train_inps_[i])
        train_oup__.extend(train_oups_[i])
    md = CNN()
    md.setup()
    train_oup__ = np.array(train_oup__)
    train_inp__ = np.array(train_inp__)
    md.setData(train_inp__,train_oup__)
    sess = getSession()
    md.train('FMNDH', 'GlobalDH-' + sess, epochs_=epochs, verbose_=2)
    return md

In [4]:
mod_clean = honestTrainGlobal(epochs, train_inp, train_oup)

Epoch 1/2
1641/1641 - 30s - loss: 0.0415 - accuracy: 0.7450
Epoch 2/2
1641/1641 - 30s - loss: 0.0280 - accuracy: 0.8292
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 6)         60        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 12)        660       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 12)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 12)          1308      
_________________________________________________________________
flatten (Flatten)            (None, 108)               0         
__

In [5]:
testMulti(test_inps, test_oups, mod_clean)

0 8818 8818
276/276 [==============================] - 1s 5ms/step - loss: 0.0266 - accuracy: 0.8358
3 1102 1102
35/35 [==============================] - 0s 8ms/step - loss: 0.0257 - accuracy: 0.8321
6 137 137
5/5 [==============================] - 0s 8ms/step - loss: 0.0297 - accuracy: 0.8248


In [ ]:
def_ratios = [0.103, 0.254, 0.410, 0.521, 0.642, 0.765]

r = False
if r == True:
    for def_ratio in def_ratios:
        train_oups_poisoned = poison(train_oups, def_ratio)
        mod_poisoned = dishonestTrain(epochs, train_inps, train_oups_poisoned)
        testMulti(test_inps, test_oups, mod_poisoned)

In [8]:
mod_clean.model.save('./models/mc', save_format='h5')